# Homework 2

**Name:** Oscar Beltran Villegas

**e-mail:** oscar.beltran7944@alumnos.udg.mx

# MODULES

In [5]:

import math
import pandas as pd
import numpy as np

# CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # Rotate vector
    def rotated(self, angle):
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        x = self.x * cos_theta - self.y * sin_theta
        y = self.x * sin_theta + self.y * cos_theta
        return Vec2d(x, y)

# FUNCTIONS

In [4]:
#####################################################################################
# Brownian motion trajectoy
#####################################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
    """
    Arguments:
        n_steps: Number of steps
        speed: Speed of the particle
        s_pos: Initial position of the particle
    Returns:
        BM_2d_df: DataFrame with x and y positions of the particle in pandas DataFrame
    """
    # Init velocity vector
    velocity =Vec2d(speed,0)
    
    # Init DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])    
    # Add initial position
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])    
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
    
    # Generate the trajectory
    for i in range(n_steps-1):        
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
    
        temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])    
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
        
    return BM_2d_df